# Launch pytorch traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan


In [4]:
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/',
          '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/' }

config={'128': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/config_128.yaml',
        '512': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/config_256.yaml'}

In [54]:
### Default dictionary
img_size=128
dict_pars={'epochs':20, 'batchsize':128, 'nodes':1,
           'seed':36723723, 'job_name':'test_128',
           'procs':8, 'time':'4:00:00','mode':'fresh', 'spec_loss': '--specloss'}

dict_pars['code_dir']=code_dirs[str(img_size)]
dict_pars['config_file']=config[str(img_size)]
print(dict_pars)

{'epochs': 20, 'batchsize': 128, 'nodes': 1, 'seed': 36723723, 'job_name': 'test_128', 'procs': 8, 'time': '4:00:00', 'mode': 'fresh', 'spec_loss': '--specloss', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/', 'config_file': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/config_128.yaml'}


In [55]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node={procs}
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

### Initial setup

conda activate v_py3
### module load pytorch-v1.4.0-gpu
##module load esslurm
echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

code_dir={code_dir}
config_file={config_file}
python $code_dir/main.py --seed {seed} {spec_loss} --config $config_file --mode {mode} --ngpu {procs} --batchsize {batchsize} --epochs {epochs}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [56]:
# bash_strg

In [57]:
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'

In [58]:
fname='batch_train.sh'
filename=staging_loc+fname
with open (filename,'w') as f:
    f.write(bash_strg)

In [59]:
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_train.sh


In [60]:
### Move to staging locations in project space:
os.chdir(staging_loc)

## Submit job to cori GPU

In [61]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time=4:00:00
#SBATCH --job-name=test_128

### Initial setup

conda activate v_py3
### module load pytorch-v1.4.0-gpu
##module load esslurm
echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/
config_file=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/main_code/config_128.yaml
python $code_dir/main.py --seed 36723723 --specloss --config $config_file --mode fresh --ngpu 8 --batchsize 128 --epochs 20

echo "--end date" `date` `date +%s`


In [62]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1029741
